# 07 GLS and Weighted Least Squares

When you know the error structure, you can do better than OLS + robust SE.

## Table of Contents
- [Why GLS/WLS?](#why-gls-wls)
- [WLS: Known Weight Structure](#wls-known-weight-structure)
- [Feasible GLS (FGLS)](#feasible-gls-fgls)
- [GLS for Autocorrelation (GLSAR)](#gls-for-autocorrelation-glsar)
- [When to Use What](#when-to-use-what)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Extensions (Optional)](#extensions-optional)
- [Reflection](#reflection)
- [Solutions (Reference)](#solutions-reference)

## Why This Notebook Matters

Notebook 04 showed that robust standard errors (HC3, HAC) protect inference when OLS assumptions fail. But robust SE are a **defensive** move: they fix your confidence intervals without improving your estimates. GLS and WLS go further -- if you can model the error structure correctly, you get **more efficient** (lower-variance) coefficient estimates.

This notebook teaches you:
- When and why WLS/GLS beats OLS + robust SE in efficiency.
- How to implement WLS with a known variance proxy.
- How to estimate the variance function when it is unknown (Feasible GLS).
- How to handle AR(1) errors with GLSAR.
- When to stick with OLS + robust SE instead.


## Prerequisites (Quick Self-Check)
- Completed Parts 00-04 (foundations + data + regression + inference).
- Familiarity with heteroskedasticity and robust SE from Notebooks 04 and 04a.
- Basic understanding of variance and weighted averages.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain when WLS/GLS is preferable to OLS + robust SE.
- You can implement WLS with a known variance proxy.
- You can implement FGLS by estimating the variance function from residuals.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Confusing WLS weights with frequency weights (they are precision weights: higher weight = lower variance).
- Using FGLS when the variance model is badly misspecified (can be worse than OLS).
- Forgetting that GLS efficiency gains require a *correct* variance model.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook's `data/sample/*` fallback.
- If results look "too good," suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/02_regression/07_gls_weighted_least_squares.md`

## How To Use This Notebook
- Work section-by-section; don't skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2-4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/02_regression/07_gls_weighted_least_squares.md`) for the math, assumptions, and deeper context.

<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.

In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT

## Goal

Learn when and how to move beyond OLS + robust standard errors to **Generalized Least Squares** (GLS) and **Weighted Least Squares** (WLS).

The key insight:
- OLS treats every observation equally.
- If some observations are noisier than others (heteroskedasticity) or errors are correlated across observations (autocorrelation), OLS is **inefficient** -- it does not make the best use of the data.
- GLS/WLS down-weights noisy observations and up-weights precise ones, producing **lower-variance** coefficient estimates.

The catch: you need a credible model of the error structure. If that model is wrong, GLS can be *worse* than OLS.

---

<a id="why-gls-wls"></a>
## 1. Why GLS/WLS?

### The efficiency argument

Consider two strategies when errors are heteroskedastic:

| Strategy | Coefficients | Standard Errors | Efficiency |
|----------|-------------|-----------------|------------|
| **OLS + robust SE (HC3)** | Consistent but not efficient | Valid (corrected for heteroskedasticity) | Lower -- treats all obs equally |
| **WLS / GLS** | Consistent *and* efficient (if variance model correct) | Valid *and* tighter | Higher -- down-weights noisy obs |

**Analogy:** Imagine averaging exam scores from two classes. Class A has 200 students (low noise per student), Class B has 10 students (high noise per student). A simple average treats both classes equally. A *weighted* average gives more weight to the larger, more precise class. That is what WLS does.

### When does the distinction matter?

- **Small samples:** efficiency gains from WLS can be substantial.
- **Strong heteroskedasticity patterns:** e.g., variance proportional to population size, income level, or time.
- **Forecasting / prediction:** tighter coefficient estimates translate to better predictions.

### When to stick with OLS + robust SE

- When you have no credible model of the variance structure.
- When the sample is large enough that efficiency gains are negligible.
- When misspecifying the variance model is a bigger risk than losing some efficiency.

---

## Load Data

We use **two** datasets in this notebook:

1. **Census county data** (cross-sectional) -- for WLS with a known variance proxy (county population).
2. **Macro quarterly data** (time series) -- for FGLS and GLSAR with heteroskedastic/autocorrelated residuals.

### Your Turn (1): Load cross-sectional (county) data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- Cross-sectional data (county level) ---
year = 2022  # TODO: set to the year you fetched
path_county = PROCESSED_DIR / f'census_county_{year}.csv'

if path_county.exists():
    df_county = pd.read_csv(path_county)
else:
    df_county = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')

# Build variables for cross-sectional regression
income = pd.to_numeric(df_county['B19013_001E'], errors='coerce')   # median household income
rent = pd.to_numeric(df_county['B25064_001E'], errors='coerce')     # median gross rent
pop = pd.to_numeric(df_county['B01003_001E'], errors='coerce')      # total population

mask = (income > 0) & (rent > 0) & (pop > 0)

df_cs = pd.DataFrame({
    'income': income[mask],
    'rent': rent[mask],
    'pop': pop[mask],
}).dropna()

df_cs['log_income'] = np.log(df_cs['income'])
df_cs['log_rent'] = np.log(df_cs['rent'])
df_cs['log_pop'] = np.log(df_cs['pop'])

print(f'Cross-sectional obs: {len(df_cs)}')
df_cs.head()

### Your Turn (2): Load time-series (macro) data

In [ ]:
# --- Time-series data (macro quarterly) ---
path_macro = PROCESSED_DIR / 'macro_quarterly.csv'

if path_macro.exists():
    df_macro = pd.read_csv(path_macro, index_col=0, parse_dates=True)
else:
    df_macro = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_ts_col = 'gdp_growth_qoq'
x_ts_cols = ['T10Y2Y_lag1']

df_ts = df_macro[[y_ts_col] + x_ts_cols].dropna().copy()

print(f'Time-series obs: {len(df_ts)}')
df_ts.tail()

---

<a id="wls-known-weight-structure"></a>
## 2. WLS: Known Weight Structure

### Goal

Fit a weighted least squares regression where the weights come from a known variable -- **county population**.

### Intuition

County-level statistics (median income, median rent) are themselves estimates from surveys. Larger counties have more respondents, so their estimates are more precise (lower sampling variability). We can model this as:

$$\text{Var}(\varepsilon_i) \propto \frac{1}{\text{pop}_i}$$

WLS uses weights $w_i = \text{pop}_i$ (inverse of variance) to give more weight to counties with lower error variance.

### `statsmodels` WLS API

```python
sm.WLS(y, X, weights=w).fit()
```

where `weights` = $1 / \text{Var}(\varepsilon_i)$. Higher weight means the observation is treated as **more precise**.

### Your Turn (1): Fit OLS baseline with HC3 robust SE

First, establish the OLS benchmark with heteroskedasticity-robust standard errors.

In [ ]:
# --- OLS baseline: log(rent) ~ log(income) ---
X_cs = sm.add_constant(df_cs[['log_income']], has_constant='add')
y_cs = df_cs['log_rent']

res_ols = sm.OLS(y_cs, X_cs).fit()
res_ols_hc3 = res_ols.get_robustcov_results(cov_type='HC3')

print('=== OLS (naive SE) ===')
print(res_ols.summary())
print()
print('=== OLS (HC3 robust SE) ===')
print(f'  log_income coef : {res_ols_hc3.params["log_income"]:.4f}')
print(f'  log_income SE   : {res_ols_hc3.bse["log_income"]:.4f}')
print(f'  log_income p    : {res_ols_hc3.pvalues["log_income"]:.4f}')

### Your Turn (2): Visualize heteroskedasticity by population

Before fitting WLS, confirm that residual variance is related to population.

In [ ]:
# TODO: Create a scatter plot of OLS residuals vs log(population).
# Look for a fan shape: do smaller counties have larger residual spread?
# Hint: res_ols.resid on y-axis, df_cs['log_pop'] on x-axis.

fig, ax = plt.subplots(figsize=(8, 5))

...

ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_xlabel('log(Population)')
ax.set_ylabel('OLS Residual')
ax.set_title('OLS Residuals vs County Population (heteroskedasticity check)')
plt.tight_layout()
plt.show()

### Your Turn (3): Fit WLS with population weights

Use `sm.WLS` with `weights = pop` (larger population = more precise = higher weight).

In [ ]:
# TODO: Fit WLS using population as the weight.
# weights = pop means Var(e_i) ~ 1/pop_i.

w = df_cs['pop']
res_wls = sm.WLS(..., ..., weights=...).fit()

print('=== WLS (population weights) ===')
print(res_wls.summary())

### Your Turn (4): Compare OLS + HC3 vs WLS side by side

The key comparison: do coefficients change? Do standard errors shrink?

In [ ]:
# TODO: Build a comparison table of coefficients and SE for OLS (naive),
# OLS (HC3), and WLS.

comparison = pd.DataFrame({
    'OLS_coef': res_ols.params,
    'OLS_SE_naive': res_ols.bse,
    'OLS_SE_HC3': res_ols_hc3.bse,
    'WLS_coef': ...,
    'WLS_SE': ...,
})

print('Coefficient and SE Comparison: OLS vs WLS')
print(comparison.round(4))
print()
print('Efficiency gain (SE ratio, WLS / OLS_HC3):')
print((comparison['WLS_SE'] / comparison['OLS_SE_HC3']).round(4))

### Interpretation Prompt

Write 2-4 sentences:
- Did the coefficient on `log_income` change between OLS and WLS? By how much?
- Did the WLS standard error shrink compared to OLS HC3? What does that mean for your confidence intervals?
- Why might WLS coefficients differ from OLS? (Hint: think about which observations get up-weighted.)

In [ ]:
# TODO: Write your interpretation.
notes = """
...
"""
print(notes)

---

<a id="feasible-gls-fgls"></a>
## 3. Feasible GLS (FGLS)

### Goal

When you do not have a natural variance proxy (like population), you can **estimate** the variance structure from OLS residuals. This is called **Feasible GLS**.

### The three-step recipe

1. **Step 1:** Fit OLS and get residuals $\hat{e}_i$.
2. **Step 2:** Model $\log(\hat{e}_i^2)$ as a function of $X$ (regress the log-squared residuals on the regressors).
3. **Step 3:** Use the predicted values $\hat{\sigma}_i^2 = \exp(\hat{g}_i)$ as estimated variances. Set WLS weights $w_i = 1 / \hat{\sigma}_i^2$.

### Why log-squared residuals?

- Squared residuals $\hat{e}_i^2$ are noisy and right-skewed.
- Taking logs stabilizes the variance and makes the auxiliary regression better-behaved.
- Exponentiating the fitted values guarantees positive variance estimates.

### Caveat

FGLS is only as good as the auxiliary variance model. If the variance model is badly wrong, FGLS can be **less** efficient than OLS. When in doubt, compare FGLS to OLS + robust SE.

We will demonstrate FGLS on the **macro quarterly** data, where GDP growth residuals may have time-varying volatility.

### Your Turn (1): Step 1 -- Fit OLS and inspect residuals

In [ ]:
# --- Step 1: OLS on macro data ---
X_ts = sm.add_constant(df_ts[x_ts_cols], has_constant='add')
y_ts = df_ts[y_ts_col]

res_ols_ts = sm.OLS(y_ts, X_ts).fit()

# Inspect residuals visually: do they show changing variance over time?
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(res_ols_ts.resid.index, res_ols_ts.resid.values, linewidth=0.8)
axes[0].axhline(0, color='red', linestyle='--', linewidth=1)
axes[0].set_title('OLS Residuals Over Time')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Residual')

axes[1].scatter(res_ols_ts.fittedvalues, res_ols_ts.resid, alpha=0.5, s=15)
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].set_title('Fitted vs Residuals')
axes[1].set_xlabel('Fitted values')
axes[1].set_ylabel('Residuals')

plt.tight_layout()
plt.show()

print(res_ols_ts.summary())

### Your Turn (2): Step 2 -- Model the variance function

Regress $\log(\hat{e}^2)$ on the regressors to estimate how variance changes with $X$.

In [ ]:
# --- Step 2: Auxiliary regression for variance ---
# Compute log(squared residuals)
log_resid_sq = np.log(res_ols_ts.resid ** 2)

# TODO: Regress log_resid_sq on X_ts (same regressors as the main model).
# This is the auxiliary variance model.
res_aux = sm.OLS(..., ...).fit()

print('=== Auxiliary Variance Model: log(e^2) ~ X ===')
print(res_aux.summary())
print()
print('If the slope is significant, variance depends on X (heteroskedasticity pattern).')

### Your Turn (3): Step 3 -- Compute FGLS weights and re-estimate

Use the predicted variances from Step 2 to construct weights, then fit WLS.

In [ ]:
# --- Step 3: FGLS (WLS with estimated weights) ---

# Predicted log-variance from the auxiliary model
log_var_hat = res_aux.fittedvalues

# TODO: Convert to variance estimates and compute weights.
# var_hat = exp(log_var_hat)
# weights = 1 / var_hat
var_hat = ...
fgls_weights = ...

# TODO: Fit WLS with the estimated weights.
res_fgls = sm.WLS(y_ts, X_ts, weights=...).fit()

print('=== FGLS Results ===')
print(res_fgls.summary())

### Your Turn (4): Compare OLS, OLS + HAC, and FGLS

In [ ]:
# TODO: Build a comparison table.
res_ols_hac = res_ols_ts.get_robustcov_results(cov_type='HAC', cov_kwds={'maxlags': 4})

comparison_ts = pd.DataFrame({
    'OLS_coef': res_ols_ts.params,
    'OLS_SE_naive': res_ols_ts.bse,
    'OLS_SE_HAC': res_ols_hac.bse,
    'FGLS_coef': ...,
    'FGLS_SE': ...,
})

print('Coefficient and SE Comparison: OLS vs FGLS (Macro Time Series)')
print(comparison_ts.round(4))
print()
print('Efficiency gain (SE ratio, FGLS / OLS_HAC):')
print((comparison_ts['FGLS_SE'] / comparison_ts['OLS_SE_HAC']).round(4))

### Interpretation Prompt

Write 2-4 sentences:
- Did the auxiliary regression find a significant variance pattern? What does that tell you?
- Did FGLS produce tighter standard errors than OLS + HAC?
- Under what circumstances would FGLS be *worse* than OLS + robust SE?

In [ ]:
# TODO: Write your interpretation.
notes = """
...
"""
print(notes)

---

<a id="gls-for-autocorrelation-glsar"></a>
## 4. GLS for Autocorrelation (GLSAR)

### Goal

When time-series residuals follow an AR(1) process, GLS can account for the serial correlation directly rather than just fixing the standard errors.

### Background

If the errors follow $\varepsilon_t = \rho \varepsilon_{t-1} + u_t$, then:
- **OLS + HAC:** Coefficients are consistent, SE are corrected. Simple and safe.
- **Cochrane-Orcutt / Prais-Winsten:** Transform the data by quasi-differencing ($y_t - \rho y_{t-1}$), then run OLS on the transformed data. More efficient if $\rho$ is correctly estimated.
- **`sm.GLSAR`:** Iterates between estimating $\rho$ from residuals and re-estimating the regression. Converges to the GLS solution.

### `statsmodels` GLSAR API

```python
from statsmodels.regression.linear_model import GLSAR

model = GLSAR(y, X, rho=1)     # rho=1 means estimate AR(1)
res = model.iterative_fit(maxiter=50)
```

### Your Turn (1): Check for AR(1) residuals in the OLS model

In [ ]:
# TODO: Compute lag-1 autocorrelation of OLS residuals.
# Hint: res_ols_ts.resid.autocorr(lag=1)

rho_hat = ...

print(f'Estimated AR(1) coefficient from OLS residuals: {rho_hat:.4f}')
print()
if abs(rho_hat) > 0.2:
    print('Substantial autocorrelation detected. GLSAR may improve efficiency.')
else:
    print('Weak autocorrelation. OLS + HAC is likely sufficient.')

### Your Turn (2): Fit GLSAR and compare to OLS

In [ ]:
from statsmodels.regression.linear_model import GLSAR

# TODO: Fit GLSAR with iterative estimation of rho.
# rho=1 tells GLSAR to estimate a single AR(1) parameter.
model_glsar = GLSAR(y_ts, X_ts, rho=1)
res_glsar = model_glsar.iterative_fit(maxiter=...)

print('=== GLSAR Results (AR(1) correction) ===')
print(f'Estimated rho: {model_glsar.rho:.4f}')
print(res_glsar.summary())

### Your Turn (3): Compare OLS, OLS + HAC, and GLSAR

In [ ]:
# TODO: Build a comparison table for all three approaches.

comparison_ar = pd.DataFrame({
    'OLS_coef': res_ols_ts.params,
    'OLS_SE_naive': res_ols_ts.bse,
    'OLS_SE_HAC': res_ols_hac.bse,
    'GLSAR_coef': ...,
    'GLSAR_SE': ...,
})

print('Coefficient and SE Comparison: OLS vs GLSAR')
print(comparison_ar.round(4))

### Interpretation Prompt

Write 2-4 sentences:
- How does the GLSAR-estimated $\rho$ compare to the residual autocorrelation you computed?
- Did GLSAR change the coefficient or mostly the standard errors?
- When would you prefer GLSAR over simply using HAC standard errors?

In [ ]:
# TODO: Write your interpretation.
notes = """
...
"""
print(notes)

---

<a id="when-to-use-what"></a>
## 5. When to Use What

### Decision Tree

```
Start: You have a regression model with potential error structure issues.
  |
  +-- Do you have a credible variance model (e.g., variance ~ population)?
  |     |
  |     +-- YES --> Use WLS with known weights.
  |     |           (Most efficient if the variance model is correct.)
  |     |
  |     +-- NO --> Do you observe a clear heteroskedasticity pattern in residuals?
  |                 |
  |                 +-- YES --> Consider FGLS (estimate variance from residuals).
  |                 |           Compare FGLS SE to OLS + robust SE as a sanity check.
  |                 |
  |                 +-- NO --> Are the residuals autocorrelated (time series)?
  |                             |
  |                             +-- YES --> Use GLSAR (AR(1) correction)
  |                             |           or OLS + HAC (safer, fewer assumptions).
  |                             |
  |                             +-- NO --> OLS + robust SE (HC3 for cross-section,
  |                                        HAC for time series).
  |                                        This is the safe default.
```

### Summary Table

| Method | Assumptions | Coefficients | SE | Efficiency | Risk |
|--------|-------------|-------------|----|-----------|----- |
| OLS + robust SE | Minimal | Consistent | Valid | Baseline | Low |
| WLS (known weights) | Variance model correct | Consistent + efficient | Valid + tight | High | Medium (if weights wrong) |
| FGLS (estimated weights) | Variance model approximately correct | Consistent + efficient | Valid + tight | Medium-High | Higher (estimated weights add noise) |
| GLSAR (AR correction) | AR(1) error structure | Consistent + efficient | Valid + tight | High (if AR(1) holds) | Medium (if AR structure wrong) |

### Rule of thumb

**Default to OLS + robust SE.** Only move to GLS/WLS when:
1. You have a strong theoretical or empirical reason for the variance model.
2. You verify that the GLS/WLS standard errors are actually tighter (not always guaranteed with estimated weights).
3. You compare the GLS coefficient to the OLS coefficient -- if they diverge substantially, investigate why.

---

<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)

Run a few asserts and write 2-3 sentences summarizing what you verified.

In [ ]:
# TODO: Validate your results. Uncomment and adjust as needed.

# Cross-sectional data loaded
# assert df_cs.shape[0] > 50, f'Too few county observations: {df_cs.shape[0]}'

# Time-series data loaded
# assert df_ts.shape[0] > 30, f'Too few time-series observations: {df_ts.shape[0]}'

# WLS produced valid results
# assert np.isfinite(res_wls.params['log_income']), 'WLS coefficient is not finite'
# assert res_wls.bse['log_income'] > 0, 'WLS SE should be positive'

# OLS and WLS coefficients should be in the same ballpark
# assert abs(res_ols.params['log_income'] - res_wls.params['log_income']) < 1.0, \
#     'OLS and WLS coefficients differ by more than 1.0 -- investigate'

# FGLS weights should all be positive
# assert (fgls_weights > 0).all(), 'FGLS weights must be positive'

# GLSAR rho should be between -1 and 1
# assert -1 < model_glsar.rho < 1, f'GLSAR rho out of range: {model_glsar.rho}'

print('All checkpoint assertions passed.')
...

---

<a id="extensions-optional"></a>
## Extensions (Optional)

- Try WLS with `weights = np.sqrt(pop)` instead of `pop`. How sensitive are the results to the weight specification?
- Add `log_pop` as a regressor to the cross-sectional model. Does the heteroskedasticity pattern change after controlling for population?
- For FGLS, try adding squared terms or additional predictors to the auxiliary variance regression. Does the fit improve?
- Fit GLSAR with `rho=2` (AR(2) errors) and compare to the AR(1) version.
- Implement Cochrane-Orcutt manually: estimate $\rho$ from OLS residuals, quasi-difference the data ($y_t - \hat{\rho} y_{t-1}$), and run OLS on the transformed data. Compare to `GLSAR`.

---

<a id="reflection"></a>
## Reflection

- What did you assume about the variance structure when fitting WLS and FGLS? How would you test those assumptions?
- If you had to ship a regression model, would you use WLS/GLS or stick with OLS + robust SE? What factors would drive that decision?
- How does the efficiency vs. robustness tradeoff in GLS/WLS parallel similar tradeoffs in other areas of statistics and ML (e.g., bias-variance tradeoff)?

---

<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: WLS -- Visualize heteroskedasticity</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- Visualize heteroskedasticity
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(df_cs['log_pop'], res_ols.resid, alpha=0.3, s=10)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_xlabel('log(Population)')
ax.set_ylabel('OLS Residual')
ax.set_title('OLS Residuals vs County Population (heteroskedasticity check)')
plt.tight_layout()
plt.show()
```

</details>

<details><summary>Solution: WLS -- Fit WLS with population weights</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- Fit WLS
w = df_cs['pop']
res_wls = sm.WLS(y_cs, X_cs, weights=w).fit()
print(res_wls.summary())
```

</details>

<details><summary>Solution: WLS -- Compare OLS vs WLS</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- Compare OLS vs WLS
comparison = pd.DataFrame({
    'OLS_coef': res_ols.params,
    'OLS_SE_naive': res_ols.bse,
    'OLS_SE_HC3': res_ols_hc3.bse,
    'WLS_coef': res_wls.params,
    'WLS_SE': res_wls.bse,
})
print(comparison.round(4))
print()
print('Efficiency gain (SE ratio, WLS / OLS_HC3):')
print((comparison['WLS_SE'] / comparison['OLS_SE_HC3']).round(4))
```

</details>

<details><summary>Solution: FGLS -- Auxiliary variance model</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- FGLS Step 2
log_resid_sq = np.log(res_ols_ts.resid ** 2)
res_aux = sm.OLS(log_resid_sq, X_ts).fit()
print(res_aux.summary())
```

</details>

<details><summary>Solution: FGLS -- Compute weights and re-estimate</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- FGLS Step 3
log_var_hat = res_aux.fittedvalues
var_hat = np.exp(log_var_hat)
fgls_weights = 1.0 / var_hat

res_fgls = sm.WLS(y_ts, X_ts, weights=fgls_weights).fit()
print(res_fgls.summary())
```

</details>

<details><summary>Solution: FGLS -- Compare OLS vs FGLS</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- Compare OLS vs FGLS
res_ols_hac = res_ols_ts.get_robustcov_results(cov_type='HAC', cov_kwds={'maxlags': 4})

comparison_ts = pd.DataFrame({
    'OLS_coef': res_ols_ts.params,
    'OLS_SE_naive': res_ols_ts.bse,
    'OLS_SE_HAC': res_ols_hac.bse,
    'FGLS_coef': res_fgls.params,
    'FGLS_SE': res_fgls.bse,
})
print(comparison_ts.round(4))
print()
print('Efficiency gain (SE ratio, FGLS / OLS_HAC):')
print((comparison_ts['FGLS_SE'] / comparison_ts['OLS_SE_HAC']).round(4))
```

</details>

<details><summary>Solution: GLSAR -- Check autocorrelation and fit</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- GLSAR
from statsmodels.regression.linear_model import GLSAR

rho_hat = res_ols_ts.resid.autocorr(lag=1)
print(f'Residual AR(1): {rho_hat:.4f}')

model_glsar = GLSAR(y_ts, X_ts, rho=1)
res_glsar = model_glsar.iterative_fit(maxiter=50)
print(f'Estimated rho: {model_glsar.rho:.4f}')
print(res_glsar.summary())
```

</details>

<details><summary>Solution: GLSAR -- Compare all methods</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 07_gls_weighted_least_squares -- Compare all methods
comparison_ar = pd.DataFrame({
    'OLS_coef': res_ols_ts.params,
    'OLS_SE_naive': res_ols_ts.bse,
    'OLS_SE_HAC': res_ols_hac.bse,
    'GLSAR_coef': res_glsar.params,
    'GLSAR_SE': res_glsar.bse,
})
print(comparison_ar.round(4))
```

</details>